# Load Dataset


In [68]:
import tensorflow as tf 
import tensorflow_datasets as tfds 
import os
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import PIL
from tensorflow.keras import layers, Model
from tensorflow.keras.layers import Dense, Layer, Conv2D, MaxPooling2D, GlobalAveragePooling2D,BatchNormalization, Reshape, Conv2DTranspose
import time

from IPython import display
from tqdm import tqdm

In [69]:
import urllib
categories = [line.rstrip(b'\n') for line in urllib.request.urlopen('https://raw.githubusercontent.com/googlecreativelab/quickdraw-dataset/master/categories.txt')]
print(categories[:10])
category = 'candle'

[b'aircraft carrier', b'airplane', b'alarm clock', b'ambulance', b'angel', b'animal migration', b'ant', b'anvil', b'apple', b'arm']


In [70]:
# Creates a folder to download the original drawings into.
# We chose to use the numpy format : 1x784 pixel vectors, with values going from 0 (white) to 255 (black). We reshape them later to 28x28 grids and normalize the pixel intensity to [-1, 1]

if not os.path.isdir('npy_files'):
    os.mkdir('npy_files')
    
url = f'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/{category}.npy'  
urllib.request.urlretrieve(url, f'npy_files/{category}.npy')

images = np.load(f'npy_files/{category}.npy')
print(f'{len(images)} images to train on')

# You can limit the amount of images you use for training by setting :
train_images = images[:10000]
# You should also define a samller subset of the images for testing..
# TODO
test_images = images[10000:12000]
print(len(test_images))
print(len(train_images))
# Notice that this to numpy format contains 1x784 pixel vectors, with values going from 0 (white) to 255 (black). We reshape them later to 28x28 grids and normalize the pixel intensity to [-1, 1]

141545 images to train on
2000
10000


In [71]:
# define shuffle and batching sizes
batch_size = 16
shuffle_size = 1000
prefetch_size = 32

In [72]:
def preprocess(data):
    # # change dtype to float32
    # data = data.map(lambda img, label: (tf.cast(img, tf.float32), label))
    # # change value range from 0-255 to -1 - 1
    # data = data.map(lambda img, label: ((img/128)-1, label))
    # # change format to 28,28,1
    # data = data.map(lambda img, label: (tf.reshape(img, (28,28,1)), label))

    data = data.map(lambda img: tf.cast(img, tf.float32))
    # change value range from 0-255 to -1 - 1
    data = data.map(lambda img: (img/128)-1)
    # change format to 28,28,1
    data = data.map(lambda img: tf.reshape(img, (28,28,1)))

    # do other prepocessing stuff
    data = data.shuffle(shuffle_size).batch(batch_size).prefetch(prefetch_size)
    return data




# Data Pipeline


In [73]:
# create datasets from tensor
labels = np.ones_like(train_images)
labels_test = np.ones_like(test_images)

# train_ds = tf.data.Dataset.from_tensor_slices((train_images,labels))
# test_ds = tf.data.Dataset.from_tensor_slices((test_images, labels_test))
train_ds = tf.data.Dataset.from_tensor_slices(train_images)
test_ds = tf.data.Dataset.from_tensor_slices(test_images)

train_ds = preprocess(train_ds)
test_ds = preprocess(test_ds)

#check data format real quick
# for x in test_ds:
#     print(x)

# Discriminator 

In [74]:
class Discriminator(Model):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.l = [
            Conv2D(32,kernel_size=2, padding='same'),
            Conv2D(32, kernel_size=3, padding='same'),
            BatchNormalization(),
            Conv2D(32, kernel_size=2, padding='same'),
            Conv2D(32, kernel_size=2, padding='same'),
            BatchNormalization(),
            Dense(1, activation='sigmoid')
        ]
        # self.conv1 = Conv2D(32,kernel_size=2, padding='same')
        # self.conv2 = Conv2D(32, kernel_size=3, padding='same')
        # self.batch1 = BatchNormalization()

        # self.conv3 = Conv2D(32, kernel_size=2, padding='same')
        # self.conv4 = Conv2D(32, kernel_size=2, padding='same')
        # self.batch2 = BatchNormalization()

        # self.out = Dense(1, activation='sigmoid')

    @tf.function
    def call(self, x, training):
        for l in self.l:
           x = l(x)
        return x 


# Generator Class


In [75]:
class Generator(Model):
    def __init__(self):
        super(Generator, self).__init__()
        self.l = [
            Dense(49, activation='relu'),
            Reshape((7,7,1)),
            Conv2DTranspose(32, kernel_size=(2,2), strides=2, activation="relu", padding='same'),
            BatchNormalization(),
            Conv2DTranspose(32, kernel_size=(2,2), strides=2, activation='relu', padding='same'),
            BatchNormalization(),
            Conv2DTranspose(32, kernel_size=(2,2), strides=2, activation='relu', padding='same'),
            BatchNormalization(),
            Conv2D(1, kernel_size=2, strides=2, activation='tanh', padding='same')
        ]


    def call(self, x, training):
        for l in self.l:
            x = l(x)
        return x

In [76]:
G_Test = Generator()
x = tf.random.normal([batch_size,7])
x = G_Test(x)
print(x)

tf.Tensor(
[[[[-0.0001925 ]
   [-0.00108132]
   [ 0.00017479]
   ...
   [ 0.        ]
   [ 0.        ]
   [ 0.        ]]

  [[ 0.00033606]
   [ 0.00107834]
   [ 0.00057652]
   ...
   [ 0.        ]
   [ 0.        ]
   [ 0.        ]]

  [[-0.00158187]
   [-0.00285763]
   [-0.00232631]
   ...
   [ 0.        ]
   [ 0.        ]
   [ 0.        ]]

  ...

  [[ 0.00176244]
   [ 0.00565496]
   [ 0.00302357]
   ...
   [ 0.        ]
   [ 0.        ]
   [ 0.        ]]

  [[-0.00829536]
   [-0.01498458]
   [-0.01219899]
   ...
   [ 0.        ]
   [ 0.        ]
   [ 0.        ]]

  [[-0.00193295]
   [ 0.00212681]
   [ 0.00067171]
   ...
   [ 0.        ]
   [ 0.        ]
   [ 0.        ]]]


 [[[ 0.        ]
   [ 0.        ]
   [ 0.        ]
   ...
   [ 0.        ]
   [ 0.        ]
   [ 0.        ]]

  [[ 0.        ]
   [ 0.        ]
   [ 0.        ]
   ...
   [ 0.        ]
   [ 0.        ]
   [ 0.        ]]

  [[ 0.        ]
   [ 0.        ]
   [ 0.        ]
   ...
   [ 0.        ]
   [ 0.        ]


In [77]:
def train_step(G,D,G_optimizer, D_optimizer, data, noise):
    with tf.GradientTape() as G_tape, tf.GradientTape() as D_tape:

        fake_data = G(noise)
        fake_data_pred = D(fake_data)
        real_data_pred = D(data)

        D_loss = -tf.math.reduce_mean( tf.math.log(real_data_pred) + tf.math.log(1-fake_data_pred) )    
        G_loss = tf.math.reduce_mean( tf.math.log(1-fake_data_pred) )

        D_gradients = D_tape.gradient(D_loss, D.trainable_variables)
        D_optimizer.apply_gradients(zip(D_gradients, D.trainable_variables))

        G_gradients = G_tape.gradient(G_loss, G.trainable_variables)
        G_optimizer.apply_gradients(zip(G_gradients, G.trainable_variables))



In [78]:
# maybe seperate test function
def test(D,G, data, noise):

    fake_data = G(noise)
    fake_data_pred = D(fake_data)
    real_data_pred = D(data)

    D_loss = -tf.math.reduce_mean( tf.math.log(real_data_pred) + tf.math.log(1-fake_data_pred) )    
    G_loss = tf.math.reduce_mean( tf.math.log(1-fake_data_pred) )

    return D_loss, G_loss

In [81]:
num_epochs = 10

test_losses = []

G = Generator()
D = Discriminator()
D_optimizer = tf.keras.optimizers.Adam()
G_optimizer = tf.keras.optimizers.Adam()

test_noise = tf.random.normal([batch_size,7])


print("Firing up Generator and Discriminator")
for epoch in range(num_epochs):       
    losses_gen = []
    losses_dis = []     
    loss_epoch_gen = []
    loss_epoch_dis = []
    for data in tqdm(train_ds):
        z = tf.random.normal([batch_size,7])
        #train_step(G,D,G_optimizer=G_optimizer, D_optimizer=D_optimizer, data=data, noise=z)

    
        with tf.GradientTape() as G_tape, tf.GradientTape() as D_tape:
        
            
            fake_data = G(z)
            fake_data_pred = D(fake_data)
            real_data_pred = D(data)
            
            D_loss = -tf.math.reduce_mean( tf.math.log(real_data_pred) + tf.math.log(1-fake_data_pred) )
            
            G_loss = tf.math.reduce_mean( tf.math.log(1-fake_data_pred) )

            losses_gen.append(G_loss)
            losses_dis.append(D_loss)
            #loss_epoch_gen.append(G_loss)
            D_gradients = D_tape.gradient(D_loss, D.trainable_variables)
            D_optimizer.apply_gradients(zip(D_gradients, D.trainable_variables))

            G_gradients = G_tape.gradient(G_loss, G.trainable_variables)
            G_optimizer.apply_gradients(zip(G_gradients, G.trainable_variables))

    loss_epoch_gen.append(tf.reduce_mean(losses_gen))
    loss_epoch_dis.append(tf.reduce_mean(losses_dis))

    print("Generator Loss for this epoch is: " + str(loss_epoch_gen))
    print("Discriminator Loss for this epoch is: " + str(loss_epoch_dis))

        


Firing up Generator and Discriminator


  0%|          | 0/625 [00:00<?, ?it/s]2022-01-14 11:54:04.120178: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-14 11:54:04.151896: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-14 11:54:04.246306: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-14 11:54:04.346127: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-14 11:54:04.498935: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
100%|██████████| 625/625 [00:23<00:00, 26.08it/s]


Generator Loss for this epoch is: [<tf.Tensor: shape=(), dtype=float32, numpy=-0.6709158>]
Discriminator Loss for this epoch is: [<tf.Tensor: shape=(), dtype=float32, numpy=1.3604752>]


100%|██████████| 625/625 [00:22<00:00, 27.76it/s]


Generator Loss for this epoch is: [<tf.Tensor: shape=(), dtype=float32, numpy=-0.6777355>]
Discriminator Loss for this epoch is: [<tf.Tensor: shape=(), dtype=float32, numpy=1.3741319>]


...

Generator Loss for this epoch is: [<tf.Tensor: shape=(), dtype=float32, numpy=-0.69037306>]
Discriminator Loss for this epoch is: [<tf.Tensor: shape=(), dtype=float32, numpy=1.3757136>]


100%|██████████| 625/625 [00:23<00:00, 26.75it/s]


Generator Loss for this epoch is: [<tf.Tensor: shape=(), dtype=float32, numpy=-0.6858362>]
Discriminator Loss for this epoch is: [<tf.Tensor: shape=(), dtype=float32, numpy=1.3664916>]


100%|██████████| 625/625 [00:24<00:00, 26.02it/s]


Generator Loss for this epoch is: [<tf.Tensor: shape=(), dtype=float32, numpy=-0.6907184>]
Discriminator Loss for this epoch is: [<tf.Tensor: shape=(), dtype=float32, numpy=1.3795111>]
